In [3]:
import pandas as pd

df = pd.read_csv('F.csv')

# Show the first 5 rows
df.head()


,productTitle,originalPrice,price,sold,tagText
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,NaN,$46.79,600,Free shipping
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,NaN,$169.72,0,Free shipping
2,Desser For Bedroom With 7 Fabric Drawers Organ...,$78.4,$39.46,7,Free shipping
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",NaN,$111.99,0,Free shipping
4,Small Unit Simple Computer Desk Household Wood...,$48.82,$21.37,1,Free shipping


In [4]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   productTitle   2000 non-null   object
 1   originalPrice  487 non-null    object
 2   price          2000 non-null   object
 3   sold           2000 non-null   int64 
 4   tagText        1997 non-null   object
dtypes: int64(1), object(4)
memory usage: 78.2+ KB


In [5]:
df.isnull().sum()


productTitle        0
originalPrice    1513
price               0
sold                0
tagText             3
dtype: int64

In [6]:
df.shape

(2000, 5)

In [7]:
# Dropping originalPrice because it has too many null values
df.drop(['originalPrice'], axis=1, inplace=True)

# removing rows with any missing values
df.dropna(inplace=True)

# updated structure
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1997 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   productTitle  1997 non-null   object
 1   price         1997 non-null   object
 2   sold          1997 non-null   int64 
 3   tagText       1997 non-null   object
dtypes: int64(1), object(3)
memory usage: 78.0+ KB


In [8]:
# Removing dollar signs and convert to float
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

# the result
df.head()


,productTitle,price,sold,tagText
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,46.79,600,Free shipping
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,169.72,0,Free shipping
2,Desser For Bedroom With 7 Fabric Drawers Organ...,39.46,7,Free shipping
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",111.99,0,Free shipping
4,Small Unit Simple Computer Desk Household Wood...,21.37,1,Free shipping


In [9]:
df['tagText'] = df['tagText'].apply(lambda x: x if x in ['Free shipping', '+Shipping: $5.09'] else 'others')

# See the new value counts
df['tagText'].value_counts()


tagText
Free shipping       1880
others               108
+Shipping: $5.09       9
Name: count, dtype: int64

In [10]:
def extract_shipping(text):
    if text == 'Free shipping':
        return 0
    elif '+Shipping: $' in text:
        return float(text.split('$')[1].replace(',', ''))
    else:
        return None  # for 'others'

df['shipping_price'] = df['tagText'].apply(extract_shipping)

# Check result
df[['tagText', 'shipping_price']].head(10)


,tagText,shipping_price
0,Free shipping,0.0
1,Free shipping,0.0
2,Free shipping,0.0
3,Free shipping,0.0
4,Free shipping,0.0
5,Free shipping,0.0
6,Free shipping,0.0
7,Free shipping,0.0
9,Free shipping,0.0
10,Free shipping,0.0


In [11]:
def extract_shipping(text):
    if text == 'Free shipping':
        return 0
    elif '+Shipping: $' in text:
        return float(text.split('$')[1].replace(',', ''))
    else:
        return None  # for 'others'

df['shipping_price'] = df['tagText'].apply(extract_shipping)

# Check result
df[['tagText', 'shipping_price']].head(10)


,tagText,shipping_price
0,Free shipping,0.0
1,Free shipping,0.0
2,Free shipping,0.0
3,Free shipping,0.0
4,Free shipping,0.0
5,Free shipping,0.0
6,Free shipping,0.0
7,Free shipping,0.0
9,Free shipping,0.0
10,Free shipping,0.0


In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['tagText_encoded'] = le.fit_transform(df['tagText'])

# Check mapping
print(dict(zip(le.classes_, le.transform(le.classes_))))


{'+Shipping: $5.09': 0, 'Free shipping': 1, 'others': 2}


In [13]:
df['final_price'] = df['price'] + df['shipping_price']


In [14]:
df.to_csv("cleaned_furniture_data.csv", index=False)
